In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/malayalam-hope-speech-dataset/malayalam_dev.csv
/kaggle/input/malayalam-hope-speech-dataset/malayalam_train.csv
/kaggle/input/malayalam-hope-speech-dataset/malayalam_test.csv


In [3]:
import pandas as pd

train = pd.read_csv("/kaggle/input/malayalam-hope-speech-dataset/malayalam_train.csv",
                    sep="\t", header=None)
dev   = pd.read_csv("/kaggle/input/malayalam-hope-speech-dataset/malayalam_dev.csv",
                    sep="\t", header=None)
test  = pd.read_csv("/kaggle/input/malayalam-hope-speech-dataset/malayalam_test.csv",
                    sep="\t", header=None)

train.columns = ["raw"]
dev.columns   = ["raw"]
test.columns  = ["raw"]


In [4]:
def clean_split(df):
    parts = df["raw"].str.split(";", expand=True)
    df["text"] = parts[0]
    df["label"] = parts[1]
    df = df.dropna(subset=["label"])
    return df[["text", "label"]]

train = clean_split(train)
dev   = clean_split(dev)
test  = clean_split(test)


In [5]:
train.head()

,text,label
0,@arya s nair may be athile karthikayude charct...,Non_hope_speech
1,വാങ്ങിയത് എന്ത് കുന്തം ആയാലും കളയാൻ പറ്റില്ലല്...,Non_hope_speech
2,മാറുമറയ്ക്കാൻ നടത്തിയ സമരം ഒരു previlege issue...,Hope_speech
3,ഇഷാനെ കൂടുതല് കെട്ടിപിടിക്കേണ്ട ഞങ്ങൾക്കറിയാം ...,Non_hope_speech
4,ഇന്ത്യ ഇസ്‌ലാമിക രാജ്യമല്ല.. ഇന്ത്യൻ ഭരണഘടന മാ...,Hope_speech


In [6]:
import re
def filter_not_malayalam(df):
    return df[~df["text"].str.contains("not-malayalam", case=False, na=False)]

train = filter_not_malayalam(train)
dev   = filter_not_malayalam(dev)
test  = filter_not_malayalam(test)

def clean_text_ml(t):
    t = re.sub(r"@\w+", "", t)   # remove @mentions
    t = re.sub(r"http\S+|www\S+", "", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t



In [7]:
# train = preprocess_malayalam(train)
# dev   = preprocess_malayalam(dev)
# test  = preprocess_malayalam(test)


In [8]:
train

,text,label
0,@arya s nair may be athile karthikayude charct...,Non_hope_speech
1,വാങ്ങിയത് എന്ത് കുന്തം ആയാലും കളയാൻ പറ്റില്ലല്...,Non_hope_speech
2,മാറുമറയ്ക്കാൻ നടത്തിയ സമരം ഒരു previlege issue...,Hope_speech
3,ഇഷാനെ കൂടുതല് കെട്ടിപിടിക്കേണ്ട ഞങ്ങൾക്കറിയാം ...,Non_hope_speech
4,ഇന്ത്യ ഇസ്‌ലാമിക രാജ്യമല്ല.. ഇന്ത്യൻ ഭരണഘടന മാ...,Hope_speech
...,...,...
8559,Evan eatha eee pottan oru vivaravumilla,Non_hope_speech
8560,Ithinu pakaramayi upayogikkan pattunna Indian ...,Non_hope_speech
8561,സാറിന് നല്ലത് വരട്ടെ,Non_hope_speech
8562,ഇങ്ങനത്തെ നല്ല നല്ല അറിവുകൾ പറഞ്ഞു തരുന്ന ഈ ഡോ...,Hope_speech


In [11]:
train["text"] = train["text"].apply(clean_text_ml)
dev["text"]   = dev["text"].apply(clean_text_ml)
test["text"]  = test["text"].apply(clean_text_ml)

In [12]:
label_map = {
    "Hope_speech": 1,
    "Non_hope_speech": 0
}

train["label"] = train["label"].map(label_map)
dev["label"]   = dev["label"].map(label_map)
test["label"]  = test["label"].map(label_map)

train = train.dropna()
dev   = dev.dropna()
test  = test.dropna()


In [15]:
train

,text,label
0,s nair may be athile karthikayude charctr bise...,0.0
1,വാങ്ങിയത് എന്ത് കുന്തം ആയാലും കളയാൻ പറ്റില്ലല്...,0.0
2,മാറുമറയ്ക്കാൻ നടത്തിയ സമരം ഒരു previlege issue...,1.0
3,ഇഷാനെ കൂടുതല് കെട്ടിപിടിക്കേണ്ട ഞങ്ങൾക്കറിയാം ...,0.0
4,ഇന്ത്യ ഇസ്‌ലാമിക രാജ്യമല്ല.. ഇന്ത്യൻ ഭരണഘടന മാ...,1.0
...,...,...
8559,Evan eatha eee pottan oru vivaravumilla,0.0
8560,Ithinu pakaramayi upayogikkan pattunna Indian ...,0.0
8561,സാറിന് നല്ലത് വരട്ടെ,0.0
8562,ഇങ്ങനത്തെ നല്ല നല്ല അറിവുകൾ പറഞ്ഞു തരുന്ന ഈ ഡോ...,1.0


In [19]:
# 1. Imports
import torch
import torch.nn as nn
import torch.nn.functional as F

# 2. Device definition  ✅ REQUIRED
device = "cuda" if torch.cuda.is_available() else "cpu"

# 3. Load embedding models
mpnet_model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    device=device
)






modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
train_texts = train["text"].tolist()
dev_texts   = dev["text"].tolist()
test_texts  = test["text"].tolist()

train_mpnet = mpnet_model.encode(train_texts, batch_size=32, show_progress_bar=True)
dev_mpnet   = mpnet_model.encode(dev_texts, batch_size=32)
test_mpnet  = mpnet_model.encode(test_texts, batch_size=32)


Batches:   0%|          | 0/246 [00:00<?, ?it/s]

In [25]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"

indic_model_path = "/kaggle/input/indic-bert/indic-bert"  # UPDATE THIS PATH

indic_tokenizer = AutoTokenizer.from_pretrained(indic_model_path)
indic_model = AutoModel.from_pretrained(indic_model_path).to(device)

indic_model.eval()
print("IndicBERT loaded")

IndicBERT loaded


In [26]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    mask = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * mask, dim=1) / torch.clamp(mask.sum(dim=1), min=1e-9)
def extract_indicbert_embeddings(texts, batch_size=32):
    all_embeddings = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]

        encoded = indic_tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors="pt"
        )

        encoded = {k: v.to(device) for k, v in encoded.items()}

        with torch.no_grad():
            output = indic_model(**encoded)

        embeddings = mean_pooling(output, encoded["attention_mask"])
        embeddings = F.normalize(embeddings, p=2, dim=1)

        all_embeddings.append(embeddings.cpu())

    return torch.cat(all_embeddings).numpy()

In [27]:
train_indic_emb = extract_indicbert_embeddings(train_texts)
dev_indic_emb   = extract_indicbert_embeddings(dev_texts)
test_indic_emb  = extract_indicbert_embeddings(test_texts)

print(train_indic_emb.shape)

(7869, 768)


In [28]:
#CNN(AGNOSTIC)
import torch
import torch.nn as nn
import torch.nn.functional as F

class HopeSpeechCNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()

        self.fc1 = nn.Linear(input_dim, input_dim)
        self.dropout = nn.Dropout(0.25)

        self.conv1 = nn.Conv1d(1, 64, kernel_size=5)
        self.pool1 = nn.MaxPool1d(4)

        self.conv2 = nn.Conv1d(64, 64, kernel_size=5)
        self.pool2 = nn.MaxPool1d(4)

        self.conv3 = nn.Conv1d(64, 64, kernel_size=5)
        self.pool3 = nn.MaxPool1d(4)

        # Dynamically compute output size
        self._dummy_forward(input_dim)

        self.fc_out = nn.Linear(self.flatten_dim, 2)

    def _dummy_forward(self, input_dim):
        with torch.no_grad():
            x = torch.zeros(1, input_dim)
            x = x.unsqueeze(1)
            x = self.pool1(F.relu(self.conv1(x)))
            x = self.pool2(F.relu(self.conv2(x)))
            x = self.pool3(F.relu(self.conv3(x)))
            self.flatten_dim = x.numel()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        x = x.unsqueeze(1)
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))

        x = x.view(x.size(0), -1)
        return self.fc_out(x)

In [29]:
from sklearn.metrics import accuracy_score, f1_score
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"

def evaluate(model, loader,device):
    model.eval()
    preds, labels = [], []

    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            logits = model(X)
            pred = torch.argmax(logits, dim=1)

            preds.extend(pred.cpu().numpy())
            labels.extend(y.cpu().numpy())

    acc = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average="macro")
    return acc, macro_f1

In [30]:
def train_model(
    model,
    train_loader,
    dev_loader,
    epochs=10,
    lr=1e-4,
    model_name="model",
    device="cuda"
):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_f1 = 0.0

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for X, y in train_loader:
            X = X.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            logits = model(X)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)

        dev_acc, dev_f1 = evaluate(model, dev_loader, device)

        print(f"\nEpoch {epoch+1}/{epochs}")
        print(f"Train Loss: {avg_loss:.4f}")
        print(f"Dev Acc   : {dev_acc:.4f}")
        print(f"Dev F1    : {dev_f1:.4f}")

        if dev_f1 > best_f1:
            best_f1 = dev_f1
            torch.save(model.state_dict(), f"{model_name}.pt")
            print(" Best model saved")

    print(f"\n Best Dev Macro-F1 ({model_name}): {best_f1:.4f}")

In [31]:
from torch.utils.data import DataLoader, TensorDataset

def make_loaders(train_emb, dev_emb, test_emb, y_train, y_dev, y_test, batch_size=32):

    train_ds = TensorDataset(
        torch.tensor(train_emb, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.long)
    )

    dev_ds = TensorDataset(
        torch.tensor(dev_emb, dtype=torch.float32),
        torch.tensor(y_dev, dtype=torch.long)
    )

    test_ds = TensorDataset(
        torch.tensor(test_emb, dtype=torch.float32),
        torch.tensor(y_test, dtype=torch.long)
    )

    return (
        DataLoader(train_ds, batch_size=batch_size, shuffle=True),
        DataLoader(dev_ds, batch_size=batch_size),
        DataLoader(test_ds, batch_size=batch_size)
    )

In [32]:
#mpnet-cnn
train_loader_mpnet, dev_loader_mpnet, test_loader_mpnet = make_loaders(
    train_mpnet, dev_mpnet, test_mpnet,
    train["label"], dev["label"], test["label"]
)

model_mpnet = HopeSpeechCNN(input_dim=768).to(device)

train_model(
    model_mpnet,
    train_loader_mpnet,
    dev_loader_mpnet,
    epochs=10,
    lr=1e-4,
    model_name="cnn_mpnet_ml",
    device=device
)



Epoch 1/10
Train Loss: 0.5089
Dev Acc   : 0.8047
Dev F1    : 0.4459
 Best model saved

Epoch 2/10
Train Loss: 0.4084
Dev Acc   : 0.8201
Dev F1    : 0.7178
 Best model saved

Epoch 3/10
Train Loss: 0.3658
Dev Acc   : 0.8541
Dev F1    : 0.7358
 Best model saved

Epoch 4/10
Train Loss: 0.3433
Dev Acc   : 0.8613
Dev F1    : 0.7456
 Best model saved

Epoch 5/10
Train Loss: 0.3213
Dev Acc   : 0.8551
Dev F1    : 0.7434

Epoch 6/10
Train Loss: 0.3021
Dev Acc   : 0.8438
Dev F1    : 0.7590
 Best model saved

Epoch 7/10
Train Loss: 0.2782
Dev Acc   : 0.8479
Dev F1    : 0.7618
 Best model saved

Epoch 8/10
Train Loss: 0.2551
Dev Acc   : 0.7811
Dev F1    : 0.7188

Epoch 9/10
Train Loss: 0.2298
Dev Acc   : 0.8417
Dev F1    : 0.7540

Epoch 10/10
Train Loss: 0.2072
Dev Acc   : 0.7544
Dev F1    : 0.6898

 Best Dev Macro-F1 (cnn_mpnet_ml): 0.7618


In [34]:
#indicbert-cnn
train_loader_indic, dev_loader_indic, test_loader_indic = make_loaders(
    train_indic_emb, dev_indic_emb, test_indic_emb,
    train["label"], dev["label"], test["label"]
)

model_indic = HopeSpeechCNN(input_dim=768).to(device)

train_model(
    model_indic,
    train_loader_indic,
    dev_loader_indic,
    epochs=10,
    lr=1e-4,
    model_name="cnn_indicbert",
    device=device
)



Epoch 1/10
Train Loss: 0.5390
Dev Acc   : 0.8047
Dev F1    : 0.4459
 Best model saved

Epoch 2/10
Train Loss: 0.5025
Dev Acc   : 0.8047
Dev F1    : 0.4459

Epoch 3/10
Train Loss: 0.4781
Dev Acc   : 0.8047
Dev F1    : 0.4459

Epoch 4/10
Train Loss: 0.4514
Dev Acc   : 0.8047
Dev F1    : 0.4459

Epoch 5/10
Train Loss: 0.4314
Dev Acc   : 0.8037
Dev F1    : 0.6573
 Best model saved

Epoch 6/10
Train Loss: 0.4201
Dev Acc   : 0.7657
Dev F1    : 0.6673
 Best model saved

Epoch 7/10
Train Loss: 0.4089
Dev Acc   : 0.7174
Dev F1    : 0.6430

Epoch 8/10
Train Loss: 0.4026
Dev Acc   : 0.7986
Dev F1    : 0.6927
 Best model saved

Epoch 9/10
Train Loss: 0.3967
Dev Acc   : 0.8068
Dev F1    : 0.6962
 Best model saved

Epoch 10/10
Train Loss: 0.3941
Dev Acc   : 0.7564
Dev F1    : 0.6718

 Best Dev Macro-F1 (cnn_indicbert): 0.6962


In [36]:
X_train_fused = np.concatenate([train_mpnet, train_indic_emb], axis=1)
X_dev_fused   = np.concatenate([dev_mpnet, dev_indic_emb], axis=1)
X_test_fused  = np.concatenate([test_mpnet, test_indic_emb], axis=1)

print(X_train_fused.shape)  # (N, 1536)


(7869, 1536)


In [37]:
model_fusion = HopeSpeechCNN(input_dim=1536).to(device)

train_loader_fus, dev_loader_fus, test_loader_fus = make_loaders(
    X_train_fused,
    X_dev_fused,
    X_test_fused,
    train["label"], dev["label"], test["label"]
)

train_model(
    model_fusion,
    train_loader_fus,
    dev_loader_fus,
    epochs=10,
    lr=1e-4,
    model_name="cnn_fusion_ml",
    device=device
)



Epoch 1/10
Train Loss: 0.5083
Dev Acc   : 0.8047
Dev F1    : 0.4459
 Best model saved

Epoch 2/10
Train Loss: 0.3990
Dev Acc   : 0.8510
Dev F1    : 0.7020
 Best model saved

Epoch 3/10
Train Loss: 0.3625
Dev Acc   : 0.8582
Dev F1    : 0.7266
 Best model saved

Epoch 4/10
Train Loss: 0.3422
Dev Acc   : 0.8356
Dev F1    : 0.7363
 Best model saved

Epoch 5/10
Train Loss: 0.3222
Dev Acc   : 0.8530
Dev F1    : 0.7515
 Best model saved

Epoch 6/10
Train Loss: 0.3040
Dev Acc   : 0.8150
Dev F1    : 0.7422

Epoch 7/10
Train Loss: 0.2840
Dev Acc   : 0.8263
Dev F1    : 0.7530
 Best model saved

Epoch 8/10
Train Loss: 0.2651
Dev Acc   : 0.8099
Dev F1    : 0.7371

Epoch 9/10
Train Loss: 0.2374
Dev Acc   : 0.8510
Dev F1    : 0.7534
 Best model saved

Epoch 10/10
Train Loss: 0.2160
Dev Acc   : 0.7595
Dev F1    : 0.6980

 Best Dev Macro-F1 (cnn_fusion_ml): 0.7534


In [39]:
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)
import torch
import numpy as np

def evaluate_on_test(model, test_loader, device):
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)

            logits = model(x)
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    f1  = f1_score(all_labels, all_preds, average="macro")
    prec = precision_score(all_labels, all_preds, average="macro")
    rec  = recall_score(all_labels, all_preds, average="macro")

    print("Accuracy      :", round(acc, 4))
    print("Macro-F1      :", round(f1, 4))
    print("Macro-Precision:", round(prec, 4))
    print("Macro-Recall  :", round(rec, 4))
    print("\nClassification Report:\n")
    print(classification_report(all_labels, all_preds, digits=4))

    return acc, f1, prec, rec


In [40]:
# MPNet TEST EVALUATION

model_mpnet = HopeSpeechCNN(input_dim=768).to(device)
model_mpnet.load_state_dict(torch.load("cnn_mpnet_ml.pt"))
model_mpnet.to(device)

evaluate_on_test(model_mpnet, test_loader_mpnet, device)

Accuracy      : 0.8741
Macro-F1      : 0.8064
Macro-Precision: 0.8009
Macro-Recall  : 0.8124

Classification Report:

              precision    recall  f1-score   support

           0     0.9269    0.9149    0.9209       776
           1     0.6749    0.7098    0.6919       193

    accuracy                         0.8741       969
   macro avg     0.8009    0.8124    0.8064       969
weighted avg     0.8767    0.8741    0.8753       969



(0.8740970072239422,
 0.8064005816924105,
 0.8008848988411426,
 0.8123965065968698)

In [41]:
# INDICBERT TEST EVALUATION

model_indic = HopeSpeechCNN(input_dim=768).to(device)
model_indic.load_state_dict(torch.load("cnn_indicbert.pt"))
model_indic.to(device)

evaluate_on_test(model_indic, test_loader_indic, device)


Accuracy      : 0.8204
Macro-F1      : 0.7163
Macro-Precision: 0.7181
Macro-Recall  : 0.7147

Classification Report:

              precision    recall  f1-score   support

           0     0.8859    0.8905    0.8882       776
           1     0.5503    0.5389    0.5445       193

    accuracy                         0.8204       969
   macro avg     0.7181    0.7147    0.7163       969
weighted avg     0.8190    0.8204    0.8197       969



(0.8204334365325078, 0.7163387124994953, 0.718080993080993, 0.7146620105763581)

In [44]:
# FUSION TEST EVALUATION
model_fusion = HopeSpeechCNN(input_dim=768*2).to(device)
model_fusion.load_state_dict(torch.load("cnn_fusion_ml.pt"))

evaluate_on_test(model_fusion, test_loader_fus, device)


Accuracy      : 0.8648
Macro-F1      : 0.7825
Macro-Precision: 0.7905
Macro-Recall  : 0.7755

Classification Report:

              precision    recall  f1-score   support

           0     0.9087    0.9240    0.9163       776
           1     0.6722    0.6269    0.6488       193

    accuracy                         0.8648       969
   macro avg     0.7905    0.7755    0.7825       969
weighted avg     0.8616    0.8648    0.8630       969



(0.8648090815273478, 0.782543747698053, 0.7904837346852556, 0.7754560386731478)

In [45]:
results = {}
def evaluate_and_store(name, model, dev_loader, test_loader, device):
    from sklearn.metrics import f1_score
    import torch

    def eval_loader(loader):
        model.eval()
        preds, labels = [], []

        with torch.no_grad():
            for x, y in loader:
                x = x.to(device)
                y = y.to(device)

                logits = model(x)
                preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
                labels.extend(y.cpu().numpy())

        return f1_score(labels, preds, average="macro")

    dev_f1  = eval_loader(dev_loader)
    test_f1 = eval_loader(test_loader)

    results[name] = {
        "Dev Macro-F1": round(dev_f1, 4),
        "Test Macro-F1": round(test_f1, 4)
    }

    print(name)
    print(" Dev Macro-F1 :", round(dev_f1, 4))
    print(" Test Macro-F1:", round(test_f1, 4))
    print("-" * 40)


In [47]:
model_mpnet = HopeSpeechCNN(input_dim=768).to(device)
model_mpnet.load_state_dict(torch.load("cnn_mpnet_ml.pt"))

evaluate_and_store(
    "MPNet-CNN",
    model_mpnet,
    dev_loader_mpnet,
    test_loader_mpnet,
    device
)


MPNet-CNN
 Dev Macro-F1 : 0.7618
 Test Macro-F1: 0.8064
----------------------------------------


In [48]:
model_indic = HopeSpeechCNN(input_dim=768).to(device)
model_indic.load_state_dict(torch.load("cnn_indicbert.pt"))

evaluate_and_store(
    "IndicBERT-CNN",
    model_indic,
    dev_loader_indic,
    test_loader_indic,
    device
)


IndicBERT-CNN
 Dev Macro-F1 : 0.6962
 Test Macro-F1: 0.7163
----------------------------------------


In [49]:
model_indic = HopeSpeechCNN(input_dim=768*2).to(device)
model_indic.load_state_dict(torch.load("/kaggle/working/cnn_fusion_ml.pt"))

evaluate_and_store(
    "FUSION-CNN",
    model_fusion,
    dev_loader_fus,
    test_loader_fus,
    device
)



FUSION-CNN
 Dev Macro-F1 : 0.7534
 Test Macro-F1: 0.7825
----------------------------------------


In [50]:
import pandas as pd

df_results = pd.DataFrame(results).T
df_results


,Dev Macro-F1,Test Macro-F1
MPNet-CNN,0.7618,0.8064
IndicBERT-CNN,0.6962,0.7163
FUSION-CNN,0.7534,0.7825
